## NSMC를 딥러닝으로 해보자!

* 불러온 데이터를 품사 태그를 붙여서 토크나이징합니다.
- 딥러닝을 사용한 keras에서 text classification에서 가장 많이 나오는 게 IMDB 분류!
- IMDB : 영문판 NSMC, 영화 리뷰를 긍정/부정(1/0)을 분류
- SVM을 사용하지않고 딥러닝 방법 중 딥러닝에서 많이 쓰이는 LSTM을 사용해보자

In [1]:
from konlpy.tag import Mecab
from konlpy.tag import Okt
import json
import os
import re
from pprint import pprint

okt = Okt()
mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")

if os.path.exists('train_docs.json'):
    with open("train_docs.json", encoding='utf-8') as f:
        train_data = json.load(f)
else:
    train_data = [(text_tokenizing(text_cleaning(line[1])), line[2]) for line in train_docs if text_tokenizing(line[1])]
    #train_data = [(text_tokenizing(line[1]), line[2]) for line in train_docs if text_tokenizing(line[1])]
    
    with open("train_docs.json", 'w', encoding='utf-8') as f:
        json.dump(train_data, f, ensure_ascii=False, indent='\t')
        
if os.path.exists('test_docs.json'):
    with open("test_docs.json", encoding='utf-8') as f:
        test_data = json.load(f)
else:
    test_data = [(text_tokenizing(text_cleaning(line[1])), line[2]) for line in test_docs if text_tokenizing(line[1])]
    #test_data = [(text_tokenizing(line[1]), line[2]) for line in test_docs if text_tokenizing(line[1])]
    with open("test_docs.json", 'w', encoding='utf-8') as f:
        json.dump(test_data, f, ensure_ascii=False, indent='\t')

pprint(train_data[0])
pprint(test_data[0])

[['짜증', '목소리'], '0']
[['평점'], '0']


In [2]:
print(train_data[:3])

[[['짜증', '목소리'], '0'], [['포스터', '초딩', '영화', '오버', '연기'], '1'], [['교도소', '이야기', '재미', '평점', '조정'], '0']]


## Deep Neural Network로 분류하기

In [3]:
# 필요한 라이브러리 불러오기
import numpy as np
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential # keras에서 모델 만들 때 sequential object를 만들고 여기에 하나씩 layer를 추가하는 방식(강사님은 쓰지 않음)
# 네트워크 코드를 짜는 방법이 여러가지가 있는데 그중에 sequential방법이 있음
# 강사님은 앞선 코드처럼 하려고하기때문에 sequential말고 다른 방식(layer마다 하나하나씩) 쓴다고함
from tensorflow.keras.preprocessing.sequence import pad_sequences 
# 실제로 RNN과 LSTM구조는 cell 단위로 weight가 넘어가는데
# text는 길이가 다르기때문에 길이 다른걸 체크해주려면 최대길이를 설정할건데 최대길이보다 text가 짧을 수도 있기때문에
# 나머지 길이를 채워야할 때 자동으로 채워주는게 pad_sequence

## Set Hyperparameter 
- Hyperparameter는 우리가 마음대로 정하면 되지만 강사님 추천은 1~2만개 이상 쓰는 게 성능이 높았대
    - 5만개 이상 넘어가면 너무 오래 걸렸대

In [4]:
max_words = 35000 # feature selection 방법 (단어가 16만개정도 있는데 이걸 feature로 모두 사용하면 너무 많으니 불필요한 데이터는 자르자는 측면)
max_len = 30 # 문서의 최대 길이
batch_size = 128 # 보통 32의 배수로 많이 씀
EPOCHS = 4 # 높을 수록 성능이 잘 나오는데 오래걸림. 실습을 위해 4번정도만 (너무 높게 잡으면 overfitting일어남. 30정도가 적당했대)

## Define Network Structure 

In [12]:
class SimpleLSTM(tf.keras.Model):
    def __init__(self):
        super().__init__() # 상속받은 keras 모델의 init을 불러와
    # __init__에 네트워크 구조를 순서대로 짠다고 생각해
    # sequence로 짤 수가 있고, layer마다 하나하나 달아서 쓸 수도 있음
        self.emb = Embedding(max_words, 100) # 하나하나의 node(LSTM 전체 구조)가 100차원짜리고 max_words 개수만큼 최대로 가질 수 있음
        self.lstm = LSTM(128, dropout=0.2, recurrent_dropout=0.2) # dropout 쓰면 overfitting이 좀 줄어듦(reguralization 방법)
        self.dense = Dense(1, activation='sigmoid') # 여기서 Dense layer는 1개 (0 또는 1 출력)
        
    def __call__(self, x, training=None, mask=None):
        # 불러와서 쓸 때
        x = self.emb(x)
        x = self.lstm(x)
        return self.dense(x)

## Data Preprocessing 

**<우리가 만들어야하는 형태>**</br></br>
- 토큰을 전부 이름으로 바꿔줘(컴퓨터는 text를 못 알아먹으니까 단어를 1:1로 숫자로 맵핑한 값)
- 최대 30개로 zero-padding해줘(앞에서부터 0으로 채우면서)

In [8]:
# SVM 때와 비슷한 형태로 만들고
x_train = [doc for doc, _ in train_data]

# keras가 사용하기 위한 형태로 Tokenizing
# text tokenizer를 쓰기 위해서 기준이 되는 데이터가 먼저 필요해서 x_train 먼저 선언해줬음
tokenizer = Tokenizer(num_words=max_words) # 최대 몇개의 단어를 불러올 건지(max_words로 정의해줬었지)
tokenizer.fit_on_texts(x_train) # tokenizer쓸 때 꼭 fit을 해줘야 해

# LSTM의 input으로 넣기 위한 변환 작업
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences([doc for doc, _ in test_data])
y_train = np.array([int(label) for _, label in train_data])
y_test = np.array([int(label) for _, label in test_data])
print(x_train[0])
# 실제로 뒤에서 해보면 숫자 타입이 안 맞아서 np.array로 바꿔줘야해(정확히는 타입 변환은 np.int로 바꿔주는 건데 가장 심플한 방법이 np.array로)

# 크기를 맞춰주기 위한 zero padding
x_train = pad_sequences(x_train, value=0, padding='pre', maxlen=max_len)
x_test = pad_sequences(x_test, value=0, padding='pre', maxlen=max_len)
print('\n', x_train[0])
# value=0 : padding할 때 0으로 채워라(default값이긴 하지만 좀 더 명시해주기위해)
# padding='pre' : 앞부터 채워라

# 학습 가능한 형태로 최종 변환.
# ds : data structure
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size) 
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
# 10000개씩 shuffle, test에서는 당연히 shuffle 필요 없겠지

[48, 222]

 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0  48 222]


[48, 222] -> 위에 ['짜증', '목소리']가 숫자로 바뀐 것

## Set Model 

In [16]:
# 모델 선언
model = SimpleLSTM()

# 모델 컴파일
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
# loss가 중요! 아무거나 쓰면 절대 안돼
# 심지어 layer가 어떤 것이고 어떻게 설정해줬냐에따라 다르게 setting을 해줘야해
# 우리는 dense layer가 1이고 sigmoid 활성함수 이용 -> binary_crossentropy를 써야해(외워! 0과 1 두개의 class로 구분할 때 쓰는 것)

## Early Stopping Callback

In [17]:
earlystopper = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1)
# 실제로 validation loss가 쭉 내려가야하는데 중간에 내려가다가 안 내려가고 계속 올라가면 training이 overfitting 되고있다는 뜻
# 이런 경우에 멈춰라 해주는 것! (메모리 낭비 방지)

## Run Model 

In [18]:
# 실행, 결과 저장.
histopy = model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS, callbacks=[earlystopper])

# Epoch가 진행됨에따라 loss는 떨어지고 ETA(accuracy)는 올라가는 게 정상!
# 만약 초반 Epoch부터 loss가 올라가고 accuracy가 떨어진다면 underfitting되고 있다는 뜻(모델 잘못 만듦) -> 빠르게 멈춰서 모델 수정

Train for 1054 steps, validate for 352 steps
Epoch 1/4
1054/1054 [==============================] - 267s 254ms/step - loss: 0.5192 - accuracy: 0.7309 - val_loss: 0.4940 - val_accuracy: 0.7512
Epoch 2/4
1054/1054 [==============================] - 264s 251ms/step - loss: 0.4537 - accuracy: 0.7783 - val_loss: 0.5012 - val_accuracy: 0.7452
Epoch 3/4
1054/1054 [==============================] - 293s 278ms/step - loss: 0.4159 - accuracy: 0.7993 - val_loss: 0.5217 - val_accuracy: 0.7463
Epoch 4/4
1054/1054 [==============================] - 250s 238ms/step - loss: 0.3806 - accuracy: 0.8168 - val_loss: 0.5660 - val_accuracy: 0.7400


In [19]:
# Model Test
score, acc = model.evaluate(x_test, y_test, batch_size=batch_size)

45034/45034 [==============================] - 9s 206us/sample - loss: 0.5660 - accuracy: 0.7400


In [20]:
print("Test Score: ", score)
print("Test Accuracy: ", acc)

Test Score:  0.5659710426003809
Test Accuracy:  0.74004084


## Loss Visualization 

In [24]:
import matplotlib.pyplot as plt
%matplotlib inline

loss = histopy.histopy['loss']
val_loss = history.history['val_loss']

plt.figure()
plt.plot(loss, 'ro-', label="train_loss")
plt.plot(val_loss, 'bo-', label="val_loss")
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch')
plt.legend(loc="best")
plt.title('Training and Validation Loss')
plt.show()

# train_loss는 쭉 떨어지고있고 val_loss는 올라가는 중
# val_loss가 올라가고있다는 건 기본적으로 over_fitting되고있다는 뜻
# 지금은 Epoch가 너무 짧아서 올라갔다가 내려가는지, 아님 계속 올라가는지 확인 불가능
# 실제로 Epoch 늘려보면 overfitting이 되고있지는 않대

AttributeError: 'History' object has no attribute 'histopy'

## Save Weight 

In [26]:
# Model weight matrix 저장.
model.save_weights("nsmc_keras_simplelstm")
# 나중에 불러올 때는 model.load_weights()